<a href="https://colab.research.google.com/github/HanhengHe/BigDataCourseProj/blob/main/bigdataproj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
RUNLARGE = True
NUMCLASS= 50 if RUNLARGE else 10

In [3]:
# GET FILE

from google.colab import drive
import zipfile

drive.mount('/content/drive/', force_remount=False)

filePath = "/content/drive/MyDrive/Dataset_large" if RUNLARGE else "/content/drive/MyDrive/Dataset_small/data1.zip"

with zipfile.ZipFile(filePath,"r") as zip_ref:
    zip_ref.extractall("targetdir")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as Data
import torch.optim as optim
import torch.nn.functional as F
from torchvision import models

In [7]:
# LOADING TRAINING DATA

lsTrainLabel = []
lsTrainData = []

trainingPath = "/content/targetdir/dataset2" if RUNLARGE else "/content/targetdir/data1"

with open(trainingPath + "/train_label.txt") as f:
  for line in f.readlines():
    expression = line.split(' ')
    lsTrainLabel.append(int(expression[1]))
    dataRGB = np.load(trainingPath + "/train/" + expression[0]).transpose((2,0,1))
    lsTrainData.append(dataRGB)
    # turn training data into grayscale data
    # lsData.append((dataRGB[:, :, 0] + dataRGB[:, :, 1] + dataRGB[:, :, 2]) / 3)
    break

INITER = torch.zeros(1, lsTrainData[0].shape[0], lsTrainData[0].shape[1], lsTrainData[0].shape[2])

print("Number of traning data: {0}".format(len(lsTrainLabel)), end='\n')
print("Shape of traning data: {0}".format(lsTrainData[0].shape), end='\n')

Number of traning data: 1
Shape of traning data: (3, 224, 224)


In [8]:
# LOAD TEST DATA

lsTestLabel = []
lsTestData = []

testingPath = "/content/targetdir/dataset2" if RUNLARGE else "/content/targetdir/data1"


with open(testingPath + "/test_label.txt") as f:
  for line in f.readlines():
    expression = line.split(' ')
    lsTestLabel.append(int(expression[1]))
    dataRGB = np.load(testingPath + "/test/" + expression[0]).transpose((2,0,1))
    lsTestData.append(dataRGB)
    # turn training data into grayscale data
    # lsData.append((dataRGB[:, :, 0] + dataRGB[:, :, 1] + dataRGB[:, :, 2]) / 3)

print("Number of testing data: {0}".format(len(lsTestLabel)), end='\n')
print("Shape of testing data: {0}".format(lsTestData[0].shape), end='\n')

Number of testing data: 2500
Shape of testing data: (3, 224, 224)


In [9]:
class AlexNetFc(nn.Module):
    def __init__(self):
        super(AlexNetFc, self).__init__()
        model_alexnet = models.alexnet()
        self.features = model_alexnet.features
        self.classifier = nn.Sequential()
        for i in range(6):
            self.classifier.add_module("classifier" + str(i), model_alexnet.classifier[i])

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x


class ResNet18Fc(nn.Module):
    def __init__(self):
        super(ResNet18Fc, self).__init__()
        model_resnet18 = models.resnet18()
        self.conv1 = model_resnet18.conv1
        self.bn1 = model_resnet18.bn1
        self.relu = model_resnet18.relu
        self.maxpool = model_resnet18.maxpool
        self.layer1 = model_resnet18.layer1
        self.layer2 = model_resnet18.layer2
        self.layer3 = model_resnet18.layer3
        self.layer4 = model_resnet18.layer4
        self.avgpool = model_resnet18.avgpool

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        return x


class ResNet34Fc(nn.Module):
    def __init__(self):
        super(ResNet34Fc, self).__init__()
        model_resnet34 = models.resnet34()
        self.conv1 = model_resnet34.conv1
        self.bn1 = model_resnet34.bn1
        self.relu = model_resnet34.relu
        self.maxpool = model_resnet34.maxpool
        self.layer1 = model_resnet34.layer1
        self.layer2 = model_resnet34.layer2
        self.layer3 = model_resnet34.layer3
        self.layer4 = model_resnet34.layer4
        self.avgpool = model_resnet34.avgpool

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        return x


class ResNet50Fc(nn.Module):
    def __init__(self):
        super(ResNet50Fc, self).__init__()
        model_resnet50 = models.resnet50()
        self.conv1 = model_resnet50.conv1
        self.bn1 = model_resnet50.bn1
        self.relu = model_resnet50.relu
        self.maxpool = model_resnet50.maxpool
        self.layer1 = model_resnet50.layer1
        self.layer2 = model_resnet50.layer2
        self.layer3 = model_resnet50.layer3
        self.layer4 = model_resnet50.layer4
        self.avgpool = model_resnet50.avgpool

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        return x


class ResNet101Fc(nn.Module):
    def __init__(self):
        super(ResNet101Fc, self).__init__()
        model_resnet101 = models.resnet101()
        self.conv1 = model_resnet101.conv1
        self.bn1 = model_resnet101.bn1
        self.relu = model_resnet101.relu
        self.maxpool = model_resnet101.maxpool
        self.layer1 = model_resnet101.layer1
        self.layer2 = model_resnet101.layer2
        self.layer3 = model_resnet101.layer3
        self.layer4 = model_resnet101.layer4
        self.avgpool = model_resnet101.avgpool

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        return x


class ResNet152Fc(nn.Module):
    def __init__(self):
        super(ResNet152Fc, self).__init__()
        model_resnet152 = models.resnet152()
        self.conv1 = model_resnet152.conv1
        self.bn1 = model_resnet152.bn1
        self.relu = model_resnet152.relu
        self.maxpool = model_resnet152.maxpool
        self.layer1 = model_resnet152.layer1
        self.layer2 = model_resnet152.layer2
        self.layer3 = model_resnet152.layer3
        self.layer4 = model_resnet152.layer4
        self.avgpool = model_resnet152.avgpool

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        return x


network_dict = {"AlexNet": AlexNetFc,
                "ResNet18": ResNet18Fc,
                "ResNet34": ResNet34Fc,
                "ResNet50": ResNet50Fc,
                "ResNet101": ResNet101Fc,
                "ResNet152": ResNet152Fc}

In [10]:
# NETWORK DEFINE MODULE

torch.cuda._initialized = True

class Network(nn.Module):
    
    def __init__(self, initer: torch.Tensor, num_classes: int, kernalType: str):
        
        super(Network, self).__init__()
        self.backbone = network_dict[kernalType]()
        self.fc = nn.Linear(self.backbone(initer).shape[1], num_classes)
        
    def forward(self, x):
        
        x = self.backbone(x)
        x = self.fc(x)
        x = F.softmax(x, dim=1)
        return x 

In [11]:
CPU = torch.device("cpu")
DEVICE = torch.device("cuda:0") if torch.cuda.is_available() else CPU
# DEVICE = CPU
print("DEVICE = %s" % (DEVICE))

DEVICE = cuda:0


In [14]:
# TRAINING MODULE 

import math

def Processing(KERNALTYPE, 
               MINIBATCH_SIZE = 128, 
               Epochs = 100, 
               MINLOSS = 0.001, 
               LEARNING_RATE = 0.01, 
               lr_diff = 1e-5, 
               bChangeLr = False
               ):

  torch.cuda.empty_cache()
  model = Network(INITER, NUMCLASS, KERNALTYPE).to(DEVICE)

  trainLabels = np.array(lsTrainLabel)
  trainSet = np.array(lsTrainData)

  trainSet = torch.from_numpy(trainSet).float()
  # trainLabels = torch.from_numpy(trainLabels)    # nll
  # torch_dataset = Data.TensorDataset(trainSet, trainLabels.long())    # nll

  trainLabels = torch.nn.functional.one_hot(torch.from_numpy(trainLabels), num_classes=NUMCLASS)  # mse
  torch_dataset = Data.TensorDataset(trainSet, trainLabels.float())  # mse


  # put the dataset into DataLoader
  loader = Data.DataLoader(
    dataset=torch_dataset,
    batch_size=MINIBATCH_SIZE,
    shuffle=True,
    num_workers=0
  )

  nSteps = math.ceil(trainLabels.shape[0] / MINIBATCH_SIZE)
  outputFrequence = 10

  # OPTIMIZER
  # optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)
  optimizer = optim.Adam(model.parameters())

  model.train()
  print("\n\nKERBALTYPE: " + KERNALTYPE, end='\n')
  print("TRAINING...", end='\n')
  lastLoss = 0
  for epoch in range(Epochs):

    curLoss = 0

    for step, (batch_x, batch_y) in enumerate(loader):
        optimizer.zero_grad()        
        output = model(batch_x.to(DEVICE))
        # loss = F.nll_loss(output, batch_y.to(DEVICE))  # nll
        loss = F.mse_loss(output, batch_y.to(DEVICE))  # mse
        loss.backward()
        optimizer.step()

        curLoss += loss.item() / nSteps
    if (epoch + 1) % outputFrequence == 0:
        if bChangeLr:
            LEARNING_RATE = LEARNING_RATE / math.pow((1 + 10 * (epoch - 1) / Epochs), 0.75)
            optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)

        strMsg = " Epoch {0}, general loss: {1}".format(epoch+1, curLoss)
        print(strMsg + " lr = {2:.5}".format(LEARNING_RATE) if bChangeLr else strMsg)

    if curLoss <= MINLOSS:
        print(" Terminated with low loss. Epoch {0}, general loss: {1}, lr = {2:.5}".format(epoch+1, curLoss, LEARNING_RATE))
        break
    
    if abs(lastLoss - curLoss) <= lr_diff:
        print(" Terminated with unchanged loss. Epoch {0}, general loss: {1}, lr = {2:.5}".format(epoch+1, curLoss, LEARNING_RATE))
        break

    lastLoss = curLoss

  # TEST MODULE

  testLabels = np.array(lsTestLabel)
  testSet = np.array(lsTestData)

  testSet = torch.from_numpy(testSet).float()
  testLabels = torch.from_numpy(testLabels).to(DEVICE)

  model.eval()
  print("TESTING...", end='\n')

  test_loss = 0
  correct = 0

  # first transform the data to dataset can be processed by torch
  # torch_dataset = Data.TensorDataset(testSet, testLabels.long())  # nll

  testLabelsOneHot = torch.nn.functional.one_hot(testLabels, num_classes=NUMCLASS)  # mse
  torch_dataset = Data.TensorDataset(testSet, testLabelsOneHot.float())  # mse

  # put the dataset into DataLoader
  loader = Data.DataLoader(
    dataset=torch_dataset,
    batch_size=MINIBATCH_SIZE,
    shuffle=False,
    num_workers=0
  )

  with torch.no_grad():
    for step, (batch_x, batch_y) in enumerate(loader):
        output = model(batch_x.to(DEVICE))
        # test_loss += F.nll_loss(output, batch_y, reduction='sum').item()  # nll
        test_loss += F.mse_loss(output, batch_y.to(DEVICE), reduction='sum').item()  # nll
        predict = output.max(1, keepdim=True)[1]  # find the prediction
        labels = batch_y.max(1, keepdim=True)[1]
        correct += predict.eq(labels.view_as(predict)).sum().item()

  testSize = testLabels.shape[0]
  test_loss /= testSize
  print(" Test: Average loss:%s, Accuracy: %s/%s (%s)"
      % (test_loss, correct, testSize, correct / testSize))

In [15]:
network_ls = ["AlexNet", "ResNet18", "ResNet34", "ResNet50", "ResNet101", "ResNet152"]
for networkType in network_ls:
  Processing(networkType)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]



KERBALTYPE: AlexNet
TRAINING...
 Terminated with unchanged loss. Epoch 3, general loss: 0.03999999910593033, lr = 0.01
TESTING...
 Test: Average loss:1.96, Accuracy: 50/2500 (0.02)


KERBALTYPE: ResNet18
TRAINING...
 Terminated with low loss. Epoch 5, general loss: 0.0001463529624743387, lr = 0.01
TESTING...
 Test: Average loss:1.9597894409179688, Accuracy: 50/2500 (0.02)


KERBALTYPE: ResNet34
TRAINING...
 Terminated with low loss. Epoch 4, general loss: 0.0007504443055950105, lr = 0.01
TESTING...
 Test: Average loss:1.96, Accuracy: 50/2500 (0.02)


KERBALTYPE: ResNet50
TRAINING...
 Terminated with low loss. Epoch 3, general loss: 2.142882635780552e-07, lr = 0.01
TESTING...
 Test: Average loss:1.95999560546875, Accuracy: 50/2500 (0.02)


KERBALTYPE: ResNet101
TRAINING...
 Terminated with low loss. Epoch 3, general loss: 3.5700227840607113e-07, lr = 0.01
TESTING...
 Test: Average loss:1.96, Accuracy: 50/2500 (0.02)


KERBALTYPE: ResNet152
TRAINING...
 Terminated with low loss. Epoch 